In [1]:
# Preamble

from astropy.table import Table, column
import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

In [2]:
import sys
sys.path.append('../../../astro_codes/')
#------------------------------------
from SFR_conversions import Mag_to_lum

## Load all of the required data:

In [ ]:
gz_data = Table.read('../matched_fits/gz_data.fits')
chang_in_matched = Table.read('../matched_fits/chang_in_matched.fits')
chang_out_matched = Table.read('../matched_fits/chang_out_matched.fits')
nsa_matched = Table.read('../matched_fits/nsa_matched.fits')
samples = Table.read('../matched_fits/samples.fits')
print('Loaded all data.')

## Create masks to keep:

#### i) M*-limited
#### ii) M*-limited spirals

In [ ]:
in_ml = ((samples['in_nsa']) & 
         (samples['in_wise']) &
         (samples['in_mass_limit']))

in_spiral = ((in_ml) & 
             (samples['in_spiral']) &
             (samples['axial_ok']))

print('{} in M*-limited sample'.format(np.sum(in_ml)))
print('w. {} in M*-limited spiral sample'.format(np.sum(in_spiral)))

#### iii) mask out low-SNR fluxes

In [ ]:
def print_sample_size(mask,name):
    y = np.sum(mask)
    t = len(mask)
    print(name + ': {}/{} galaxies ({}%)'.format(y,t,np.round(100*y/t,decimals=1)))

w3_ok = chang_out_matched['flag_w3'] == 1
w4_ok = chang_out_matched['flag_w4'] == 1

def snr_mask(column,min_snr=2):
    snr = nsa_matched['PETROFLUX'][:,column]/((nsa_matched['PETROFLUX_IVAR'][:,column])**(-1/2))
    detected = nsa_matched['AMIVAR'][:,column] != 0
    ok = (snr >= min_snr) & (detected)
    return ok

fuv_ok,nuv_ok,u_ok,g_ok,r_ok,i_ok,z_ok = [snr_mask(c) for c in range(7)]

print_sample_size(w3_ok,'W3 detected')
print_sample_size(w4_ok,'W4 detected')
print_sample_size(fuv_ok,'FUV detected')
print_sample_size(nuv_ok,'NUV detected')
print_sample_size(u_ok,'u detected')

mask_bad_fluxes = Table()
mask_bad_fluxes['fuv'] = fuv_ok
mask_bad_fluxes['nuv'] = nuv_ok
mask_bad_fluxes['u'] = u_ok
mask_bad_fluxes['g'] = g_ok
mask_bad_fluxes['r'] = r_ok
mask_bad_fluxes['i'] = i_ok
mask_bad_fluxes['z'] = z_ok
mask_bad_fluxes['w3'] = w3_ok
mask_bad_fluxes['w4'] = w4_ok

In [ ]:
from bar_plots import comparison_bar_chart
#help(comparison_bar_chart)

# make the table
detection_table = Table()
column_totals = []

labels = ['1','2','3','4','5+']

arm_number = samples['m']

for m in range(5):
    in_m = (arm_number == m) & (in_spiral)
    detection_data = []
    detection_data.append(np.sum(mask_bad_fluxes['fuv'][in_m]))
    detection_data.append(np.sum(mask_bad_fluxes['nuv'][in_m]))
    detection_data.append(np.sum(mask_bad_fluxes['w3'][in_m]))
    detection_data.append(np.sum(mask_bad_fluxes['w4'][in_m]))
    detection_table[labels[m]] = detection_data
    column_totals.append(np.sum(in_m))

f,f_low,f_high = comparison_bar_chart(detection_table,labels=['FUV','NUV','W3','W4'],
                                      normalise='percent',colors='mbyr',
                                      column_totals=column_totals,linewidth=1)
plt.xlabel('arm number')
plt.ylabel('fraction detected ($\%$)')

In [ ]:
ok_all = ((mask_bad_fluxes['fuv']) & (mask_bad_fluxes['nuv']) & (mask_bad_fluxes['u']))
# 'Good' detections in the FUV, NUV and u filters.

nuv_u = (nsa_matched['M_corrected'][:,1] - nsa_matched['M_corrected'][:,2])[(in_ml) & (ok_all)]
fuv_Mag = nsa_matched['M_corrected'][:,0][(in_ml) & (ok_all)]
mass = 10**chang_out_matched['lmass50_all'][(in_ml) & (ok_all)]

nuv_u_bins, _, bins = binned_statistic(nuv_u,nuv_u,bins=np.linspace(0.5,3.5,21),
                                       statistic='median')

fuv_lum = (Mag_to_lum(fuv_Mag)/3.828e33)*1.987e15
ML = mass/fuv_lum.value
logML = np.log10(ML)

## Plot $\log(M*/L_{FUV})$ as a function of $u-r$ colour (both should be proxies for sSFR)

In [ ]:
from scipy.stats import binned_statistic

def plot_bins(data,i_bins,bins,color='k',alpha=0.5,xlabel=None,ylabel=None,plot=True):
    bin_lower = []
    bin_median = []
    bin_upper = []
    for b, i in enumerate(i_bins):
        select = bins == b+1
        bin_lower.append(np.percentile(data[select],16))
        bin_median.append(np.median(data[select]))
        bin_upper.append(np.percentile(data[select],84))
    bin_lower, bin_median, bin_upper = [np.array(u) for u in [bin_lower, bin_median, bin_upper]]
    if plot == True:
        _ = plt.plot(i_bins,bin_lower,linestyle='dashed',
                     color=color,lw=1.5)
        #_ = plt.plot(i_bins,bin_median,lw=3,color=color)
        _ = plt.plot(i_bins,bin_upper,linestyle='dashed',
                     color=color,lw=1.5)
    
    return bin_lower, bin_median, bin_upper

from palettable.colorbrewer.sequential import Greens_9, Reds_9, Blues_9

def heatmap(x,y,x_bins,y_bins,colormap,colorbar=True,log_scale=False,
            xlabel='x',ylabel='y',zlabel='z'):
    if log_scale == True:
        h = plt.hist2d(x,y,bins=[x_bins,y_bins],
                       norm=mpl.colors.LogNorm(),cmap=colormap)
    else:
        h = plt.hist2d(x,y,bins=[x_bins,y_bins],cmap=colormap)
    if colorbar == True:
        cbar = plt.colorbar()
        cbar.set_label(zlabel,rotation=270)
        cbar.ax.get_yaxis().labelpad = 20
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    return h

In [ ]:
from scipy.optimize import curve_fit

def func(x,a,b,c):
    return a*x**2 + b*x + c

xlims = (0.5,3.5)
ylims = (0,3)
xlabel = '$NUV-u$'
ylabel = '$\log(M_*/L_{FUV})$'
zlabel = '$N_\mathrm{gal}$'

fig = plt.figure(figsize=(10,8))
_ = plt.axis((xlims[0],xlims[1],ylims[0],ylims[1]))

x = nuv_u
y = logML
_ = plot_bins(y,nuv_u_bins,bins,color='k')

x_bins = np.linspace(xlims[0],xlims[1],101)
y_bins = np.linspace(ylims[0],ylims[1],101)
_ = heatmap(x,y,x_bins,y_bins,Blues_9.mpl_colormap,log_scale=True,
            xlabel=xlabel,ylabel=ylabel,zlabel=zlabel)

p,_ = curve_fit(func,x,y)

func_max = -p[1]/(2*p[0])
x_guide = np.linspace(xlims[0],xlims[1],1000)
y_guide = func(x_guide,*p)
_ = plt.plot(x_guide,y_guide,'k-',lw=3)

#plt.savefig('../output_figures/colour_vs_ML_FUV.pdf')

In [ ]:
scatter = np.abs(y - func(x,*p))
_ = plt.hist(scatter,bins=np.linspace(0,1,51),histtype='stepfilled',alpha=0.5)
print('Median scatter = {}'.format(np.round(np.median(scatter),decimals=2)))

## Estimate the FUV 'completeness':

In [ ]:
has_nuv_u = (mask_bad_fluxes['nuv']) & (mask_bad_fluxes['u'])

nuv_u = (nsa_matched['M_corrected'][:,1] - nsa_matched['M_corrected'][:,2])[(in_ml) & (has_nuv_u)]
has_fuv = mask_bad_fluxes['fuv'][(in_ml) & (has_nuv_u)]

nuv_u_bins, _, bins = binned_statistic(nuv_u,nuv_u,bins=np.linspace(0.5,3.5,21),
                                             statistic='median')

in_bins = []
in_fuvs = []
for b,_ in enumerate(nuv_u_bins):
    in_bin = bins == b
    in_bins.append(np.sum(in_bin))
    in_fuvs.append(np.sum(has_fuv[in_bin]))
in_bins, in_fuvs = [np.array(u) for u in [in_bins, in_fuvs]]
fracs = in_fuvs/in_bins

In [ ]:
from match_distributions import get_kde

full_sample = nuv_u
fuv_sample = nuv_u[has_fuv]

k_full, bandwidth = get_kde(full_sample,(0,4))
k_fuv, _ = get_kde(fuv_sample,(0,4),bandwidth=bandwidth)

In [ ]:
x_guide = np.linspace(0,4,1000)[:,np.newaxis]
y_full = np.exp(k_full.score_samples(x_guide))*len(full_sample)/4
y_fuv = np.exp(k_fuv.score_samples(x_guide))*len(fuv_sample)/4
fracs = y_fuv/y_full

fig, axarr = plt.subplots(2,1,figsize=(10,10),sharex=True)
plt.subplots_adjust(hspace=0,wspace=0)

# histogram:
plt.sca(axarr[0])
plt.fill_between(x_guide[:,0],0,y_full,color='b',alpha=0.4)
plt.plot([], [], color='b',alpha=0.4,lw=10,label='NUV detected')
plt.plot(x_guide[:,0],y_fuv,color='k',lw=2,label='FUV detected')

plt.legend(fontsize=15,loc='upper left')
_ = plt.axis((0.25,3.75,0,3000))
plt.ylabel('$N_\mathrm{gal}$')

# fractional plot
plt.sca(axarr[1])
plt.plot(x_guide[:,0],fracs,'r-',lw=3)
plt.ylabel('$f$(FUV detected)')
plt.xlabel('$NUV - u$')

In [ ]:
def func_xy(func,params):
    x_set = np.arange(0.5,3.5,0.001)
    xy = Table()
    xy['x'] = x_set
    xy['y'] = func(x_set,*params)
    return xy

from sklearn.neighbors import KDTree

def find_nearest(x,y):
    x, y = [x[:,np.newaxis],y[:,np.newaxis]]
    kdtree = KDTree(x,leaf_size=30,metric='euclidean')
    indices = kdtree.query(y,k=1,return_distance=False)
    return indices


def find_nearest_x(y,xy):
    indices = find_nearest(xy['y'],y)
    return xy['x'][indices]


def get_weight(fuv_Mag,mass,xy,k1,N1,k2,N2,
               mass_log=True):
    
    if mass_log == True:
        mass = 10**(mass)
    logmass = np.log10(mass)
    # convert mags --> lum. --> logML
    fuv_lum = (Mag_to_lum(fuv_Mag)/3.828e33)*1.987e15
    ML = mass/fuv_lum.value
    logML = np.log10(ML)
    nuv_u = find_nearest_x(logML,xy)#[:,np.newaxis]
    y1 = np.exp(k1.score_samples(nuv_u))*N1/4
    y2 = np.exp(k2.score_samples(nuv_u))*N2/4
    weights = (y1/y2)
    return weights, logML, nuv_u

# For each value of FUV, we should now be able to measure a 'weighting' to give to that point.
# If we only see the 'tail' of a given distribution, this should be more clear.

In [ ]:
fuvs = np.linspace(-13,-18,1000)
masses = np.ones(len(fuvs))*10.5

k1 = k_full
k2 = k_fuv
N1 = len(full_sample)
N2 = len(fuv_sample)
fuvs = nsa_matched['M_corrected'][:,0]
masses = chang_out_matched['lmass50_all']

in_sample = (mask_bad_fluxes['fuv']) & (chang_out_matched['lmass50_all'] >= 7)

weights, logml, nuvu = get_weight(fuvs[in_sample],masses[in_sample]
                                  ,xy,k_full,N1,k_fuv,N2)

In [ ]:
fuv_weights = np.ones(len(gz_data))*(-999)
fuv_weights[in_sample] = weights
mask_bad_fluxes['fuv_weights'] = fuv_weights

## Finally, save the 'bad flux' and 'FUV weights' table for later use.

In [ ]:
mask_bad_fluxes.write('../matched_fits/bad_flux_mask.fits',overwrite=True)

In [ ]:
ok_fuv = mask_bad_fluxes['fuv']
ok = (ok_fuv) & (in_spiral)

fuv_lum = ((Mag_to_lum(nsa_matched['M_corrected'][:,0])/3.828e33)*1.987e15)[ok]

log_fuv_lum = np.log10(fuv_lum.value)
weights = mask_bad_fluxes['fuv_weights'][ok]

In [ ]:
_ = plt.hist(log_fuv_lum,bins=np.linspace(7.5,10.5,31),alpha=0.4,
             histtype='stepfilled',label='Unweighted')
_ = plt.hist(log_fuv_lum,bins=np.linspace(7.5,10.5,31),weights=weights,
             histtype='step',color='k',lw=2,label='Weighted')

plt.xlabel('$\log(L_{FUV})$')
plt.ylabel('$N_{gal}$')
plt.legend(loc='upper left')